In [71]:
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
from torchmetrics.classification import Accuracy
import torchmetrics

import torchvision.models as models

device= torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
!pip install torchmetrics

In [5]:
import zipfile
with zipfile.ZipFile("tomato_data.zip", 'r') as zip:
  zip.extractall("tomato_data")

In [6]:
path= '/content/tomato_data'

In [101]:
train_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),  #Removed RandomCrop(32) as it resized the image back
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))   #pretrained ImageNet values
])

test_transform= transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [102]:
train_path= '/content/tomato_data/data/Training_set'
test_path= '/content/tomato_data/data/Testing_set'

train_data= datasets.ImageFolder(root=train_path, transform= train_transform)
test_set= datasets.ImageFolder(root=test_path, transform= test_transform)

for (img, label) in train_data:
  print(img.shape)
  break

torch.Size([3, 128, 128])


In [103]:
train_size = int(0.85 * len(train_data))
val_size   = len(train_data) - train_size

train_set, val_set = random_split(train_data, [train_size, val_size], generator=torch.Generator().manual_seed(42))


In [104]:
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_set, batch_size=64, shuffle=False)
test_loader  = DataLoader(test_set,  batch_size=64, shuffle=False)


for (img, label) in train_loader:
  print(img.shape)
  break

torch.Size([64, 3, 128, 128])


In [108]:
class CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CNN, self).__init__()

        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),  # 64x64x64

            # Block 2
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),  # 128x32x32

            # Block 3
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),  # 256x16x16

            # Block 4
            nn.Conv2d(256, 512, 3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1, 1))  # 512x1x1
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


In [109]:
model= CNN().to(device)

In [110]:

loss_function= nn.CrossEntropyLoss()

In [111]:
optimizer= torch.optim.Adam(model.parameters(), lr= 0.001)

scheduler= torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode= "max", factor= 0.5, patience= 3)


In [112]:
def train(dataloader, model, loss_function, optimizer):
  model.train()
  total_loss= 0

  for batch, (image, label) in enumerate(dataloader):
    image= image.to(device)
    label= label.to(device)

    prediction= model(image)
    loss= loss_function(prediction, label)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    total_loss+= loss

  avg_loss= total_loss / len(dataloader)
  print(f"Training Average Loss: {avg_loss:.4f}")

In [113]:
val_accuracy = Accuracy(task="multiclass", num_classes=10).to(device)

def validate(dataloader, model, loss_function):
    model.eval()
    total_loss = 0
    val_accuracy.reset()

    with torch.no_grad():
        for image, label in dataloader:
            image = image.to(device)
            label = label.to(device)

            pred = model(image)
            loss = loss_function(pred, label)
            total_loss += loss

            val_accuracy.update(pred, label)

    avg_loss = total_loss / len(dataloader)
    accuracy = val_accuracy.compute() * 100  # Convert to %

    print(f"Validation Loss: {avg_loss:.4f}, Validation Accuracy: {accuracy:.2f}%")

    return accuracy


In [114]:
test_accuracy = Accuracy(task="multiclass", num_classes=10).to(device)

def test(dataloader, model, loss_function):
    model.eval()
    total_loss = 0
    test_accuracy.reset()   #Reset metric state for each epoch

    with torch.no_grad():
        for image, label in dataloader:
            image = image.to(device)
            label = label.to(device)

            pred = model(image)
            loss = loss_function(pred, label)
            total_loss += loss

            test_accuracy.update(pred, label)

    avg_loss = total_loss / len(dataloader)
    accuracy = test_accuracy.compute() * 100  # Convert to %

    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}%")

    return accuracy

In [116]:
epochs = 40
for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    train(train_loader, model, loss_function, optimizer)
    val_acc = validate(val_loader, model, loss_function)
    scheduler.step(val_acc)
print("\n")
print("-----------------------------------------------------------------")
print("\n")
test(test_loader, model, loss_function)


Epoch 1/40
Training Average Loss: 0.9369
Validation Loss: 1.1171, Validation Accuracy: 56.18%

Epoch 2/40
Training Average Loss: 0.9855
Validation Loss: 1.1131, Validation Accuracy: 56.83%

Epoch 3/40
Training Average Loss: 0.9526
Validation Loss: 1.1077, Validation Accuracy: 58.26%

Epoch 4/40
Training Average Loss: 0.9475
Validation Loss: 1.1150, Validation Accuracy: 57.09%

Epoch 5/40
Training Average Loss: 0.9755
Validation Loss: 1.1043, Validation Accuracy: 58.91%

Epoch 6/40
Training Average Loss: 0.9831
Validation Loss: 1.1357, Validation Accuracy: 58.13%

Epoch 7/40
Training Average Loss: 0.9340
Validation Loss: 1.1071, Validation Accuracy: 57.48%

Epoch 8/40
Training Average Loss: 0.9624
Validation Loss: 1.1326, Validation Accuracy: 58.26%

Epoch 9/40
Training Average Loss: 1.0425
Validation Loss: 1.2313, Validation Accuracy: 58.26%

Epoch 10/40
Training Average Loss: 0.9485
Validation Loss: 1.1309, Validation Accuracy: 56.05%

Epoch 11/40
Training Average Loss: 0.9621
Valida

tensor(59.6078, device='cuda:0')